In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sn
from tabulate import tabulate
import datetime
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from scipy.optimize import minimize
import matplotlib.patches as mpatches
import plotly.graph_objects as go
from scipy import stats

### Creating a dataframe with the daily closing prices of all ETFs 

In [2]:
df=pd.read_csv("C:\\Users\\kabra\\OneDrive - SP JAIN SCHOOL OF GLOBAL MANAGEMENT\\Desktop\\umass\\MasterFile9July.csv")
df=df.rename(columns = {'Unnamed: 0':'stock_name'})
df=df.drop('Unnamed: 1',axis=1)
df.Date=pd.to_datetime(df.Date)
dfclose = df[['Date', 'stock_name', 'Close']]
closing = dfclose.set_index('Date')
closing = closing.pivot(columns='stock_name')
closing.columns = [col[1] for col in closing.columns]
closing

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
Date,,,,,,,,,
1998-12-22,12.811,14.369,9.497,15.749,24.93,16.270,13.906,18.409,19.647
1998-12-23,12.946,14.669,9.643,16.024,25.54,16.654,13.855,18.823,19.719
1998-12-24,13.244,14.591,9.707,16.224,25.44,16.644,14.110,18.938,20.090
1998-12-28,13.128,14.514,9.586,16.330,25.51,16.539,14.037,18.662,19.912
1998-12-29,13.369,14.659,9.681,16.552,25.57,16.903,14.277,19.076,20.350
...,...,...,...,...,...,...,...,...,...
2020-07-02,57.520,37.340,22.975,68.990,105.27,58.990,57.920,101.590,129.540
2020-07-06,58.360,37.450,23.420,69.980,107.10,59.270,57.180,102.630,132.580
2020-07-07,58.360,36.260,22.940,68.640,106.03,59.830,56.960,101.730,130.430


### calculating the daily closing returns of all the ETFs

In [3]:
c2c=closing.pct_change()
c2c=c2c.dropna()
c2c.describe()

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
count,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000,5411.000000
mean,0.000398,0.000326,0.000349,0.000461,0.000407,0.000289,0.000338,0.000385,0.000599
std,0.015601,0.018107,0.019488,0.020700,0.016514,0.009987,0.012531,0.011848,0.024622
min,-0.115030,-0.201316,-0.164703,-0.487058,-0.138241,-0.093645,-0.113680,-0.099378,-0.582112
25%,-0.007049,-0.008211,-0.007110,-0.005725,-0.006382,-0.004327,-0.005481,-0.005160,-0.005776
50%,0.000657,0.000511,0.000479,0.000704,0.000931,0.000455,0.000946,0.000692,0.000783
75%,0.008193,0.009561,0.007862,0.007022,0.007579,0.005141,0.006610,0.006313,0.007249
max,0.121800,0.202932,0.307198,0.962298,0.171466,0.093349,0.126270,0.131194,1.359243


### calculating the daily equal-weighted portfolio 

In [4]:
eq1 = pd.DataFrame(c2c.mean(axis = 1))
eq1.columns = ['EQ']
eq1

,EQ
Date,
1998-12-23,0.014979
1998-12-24,0.008404
1998-12-28,-0.005792
1998-12-29,0.015277
1998-12-30,-0.006703
...,...
2020-07-02,0.007264
2020-07-06,0.010481
2020-07-07,-0.011199
